# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [254]:
def date_sorter(): 
    tst1 = df.str.extractall(r"((?:\d{2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:[\s\.,]{,2}\d{4}))")
    tst1 = tst1.replace('Janaury', 'January', regex=True).replace('Decemeber', 'December', regex=True)
    dates1 = pd.to_datetime(tst1[0])
    index_left1 = ~df.index.isin([x[0] for x in tst1.index])

    tst2 = df[index_left1].str.extractall(r"((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\.\s\-]{1}[\s]?\d{1,2}[-,\s][\s]?\d{4})")
    tst0 = tst1.append(tst2)
    dates2 = pd.to_datetime(tst2[0])
    index_left2 = ~df.index.isin([x[0] for x in tst0.index])

    tst3 = df[index_left2].str.extractall(r"((?:\d{0,2}/\d{0,2}/(?:\d{2}))\b|((?:\d{0,2}-\d{0,2}-(?:\d{2})))\b)")
    tstx = tst0.append(tst3)
    dates3 = pd.to_datetime(tst3[0])
    dates3 = dates3.dt.strftime('%Y-%m-%d').replace(r'^20', '19', regex=True)
    index_left3 = ~df.index.isin([x[0] for x in tstx.index])

    tst4 = df[index_left3].str.extractall(r'((?:\d{0,2}/?\d{0,2}/?[12]{1}\d{3})|(?:\d{0,2}-?\d{0,2}-?(?:[12]{1}\d{3}))|(?:[12]{1}\d{3}))')
    tstz = tstx.append(tst4)
    dates4 = pd.to_datetime(tst4[0])
    index_left4 = ~df.index.isin([x[0] for x in tstz.index])

    #tst5 = df[index_left4].str.extractall(r'([12]{1}\d{3})')
    #tsty = tstz.append(tst5)
    #dates5 = pd.to_datetime(tst5[0])
    #index_left5 = ~df.index.isin([x[0] for x in tsty.index])

    dates = dates1.append(dates2).append(dates3).append(dates4)#.append(dates5)

    dates = pd.to_datetime(dates)
    dates_final = dates.reset_index().set_index('level_0').drop(columns=['match'])
    dates_final['index'] = dates_final.index
    dates_final = dates_final.sort_values(by=[0,'index'])
    return dates_final.reset_index()['index']

date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64

In [ ]:
df.iloc[323]

In [ ]:
tst4.reset_index().set_index('level_0').loc[323]

In [255]:
import numpy as np
s_test = date_sorter()

# check if running the code twice produces the same result
try:
    assert (date_sorter() == s_test).all()
    print("Passed repeatability check")
except:
    print("Failed repeatability check")

# check if the result has the expected index
try:
    assert type(date_sorter().index) == pd.RangeIndex
    assert (date_sorter().index == pd.RangeIndex(start=0, stop=500, step=1)).all()
    print("Passed index check")
except:
    print("Failed index check")

# check the tie-break sort for a sample of records where some have the same date
# note that this only tests a sample and does not check the entire answer
try:
    i_test = [s_test.index[s_test == v].values[0]
              for v in [318, 369, 493, 252, 314, 410, 490]]
    assert sorted(i_test) == i_test
    print("Passed secondary sort sample check")
except:
    print("Failed secondary sort sample check")

# check if the parsed dates appear to be correct and correctly sorted
# by producing some test checksums
# if you get for example a False entry in the agree column for
# index value 20 that would mean you have at least one incorrectly
# parsed or incorrectly sorted date in the **output** index
# range 20,21,...,29
try:
    v_check = pd.DataFrame({'correct':
    [6695, 14428, 16742, 9275, 12290, 14654, 9421, 10185, 11464, 16491,
     11797, 14036, 15459, 9412, 13069, 10400, 10498, 14322, 13274, 11001,
     11383, 11910, 10977, 9692, 10199, 10187, 15456, 13491, 9186, 13646,
     11142, 13724, 10994, 12905, 15968, 16648, 13966, 14607, 16932, 14622,
     17942, 18220, 17818, 18305, 19633, 12522, 13978, 18445, 20156, 14797],
    'learner':[
    (s_test.iloc[10*i:(i+1)*10].values * np.array(range(1,11))).sum() for i in range(50)]},
    index=range(0,500,10)).assign(agree=lambda x:x['correct']==x['learner'])
    print("Values checksums:")
    print(v_check)
    assert v_check['agree'].all()
    print("Passed values check")
except:
    print("Failed values check")

Passed repeatability check
Passed index check
Passed secondary sort sample check
Values checksums:
     correct  learner  agree
0       6695     6695   True
10     14428    14428   True
20     16742    16742   True
30      9275     9275   True
40     12290    12290   True
50     14654    14654   True
60      9421     9421   True
70     10185    10185   True
80     11464    11464   True
90     16491    16491   True
100    11797    11797   True
110    14036    14036   True
120    15459    15459   True
130     9412     9412   True
140    13069    13069   True
150    10400    10400   True
160    10498    10498   True
170    14322    14322   True
180    13274    13274   True
190    11001    11001   True
200    11383    11383   True
210    11910    11910   True
220    10977    10977   True
230     9692     9692   True
240    10199    10199   True
250    10187    10187   True
260    15456    15456   True
270    13491    13491   True
280     9186     9186   True
290    13646    13646   True
30